In [59]:
import os
import cv2
import numpy as np
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard
import time

In [60]:
people = ["Ale", "Mauricio Vargas", "Sebastian Rojas", "Ignacio Valencia"]

def create_label_dict(names, size):
    label_dict = {}
    for name in names:
        label = np.zeros(size)
        label[names.index(name)] = 1
        label_dict[name] = label
    return label_dict

size_people = len(people)
name_pos_dict = create_label_dict(people, size_people)

def my_data():
    data = []
    for img in tqdm(os.listdir("images")):
        path = os.path.join("images", img)
        img_name = img.split('.')[0]
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        try:
            img_data = cv2.resize(img_data, (50, 50))
        except Exception as e:
            print(e)
        if img_name in name_pos_dict:
            data.append([np.array(img_data), name_pos_dict[img_name]])
    shuffle(data)
    return data

data = my_data()
len(data)

train_percentage = 0.7
split_point = int(len(data) * train_percentage)

train = data[:split_point]
test = data[split_point:]
X_train = np.array([i[0] for i in train]).reshape(-1, 50, 50, 1)
print(X_train.shape)
y_train = np.array([i[1] for i in train]) 
X_test = np.array([i[0] for i in test]).reshape(-1, 50, 50, 1)
print(X_test.shape)
y_test = np.array([i[1] for i in test]) 


 69%|██████▉   | 344/497 [00:00<00:00, 1733.25it/s]

OpenCV(4.8.1) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



100%|██████████| 497/497 [00:00<00:00, 1723.44it/s]

(277, 50, 50, 1)
(120, 50, 50, 1)


In [61]:
model_name = "model_upb_4_{}".format(int(time.time()))

log_dir = "tensor-logs/{}".format(model_name) 

if not os.path.exists(log_dir):
    os.makedirs(log_dir)

model = keras.Sequential()

model.add(keras.layers.Conv2D(32, (5, 5), activation='relu', input_shape=(50, 50, 1)))
model.add(keras.layers.MaxPooling2D(pool_size=(5, 5)))
model.add(keras.layers.Conv2D(64, (5, 5), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(5, 5)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(size_people, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

tensorboard_callback = TensorBoard(log_dir=log_dir)

model.fit(X_train, y_train, epochs=6, validation_data=(X_test, y_test), verbose=1, callbacks=[tensorboard_callback])

model.save("model_upb_4.h5")

Epoch 1/6
9/9 [==============================] - 2s 58ms/step - loss: 8.0325 - accuracy: 0.2960 - val_loss: 1.7926 - val_accuracy: 0.4250
Epoch 2/6
9/9 [==============================] - 0s 30ms/step - loss: 1.1397 - accuracy: 0.5632 - val_loss: 0.8703 - val_accuracy: 0.5500
Epoch 3/6
9/9 [==============================] - 0s 33ms/step - loss: 0.7078 - accuracy: 0.7076 - val_loss: 0.4480 - val_accuracy: 0.7750
Epoch 4/6
9/9 [==============================] - 0s 31ms/step - loss: 0.3916 - accuracy: 0.8195 - val_loss: 0.3441 - val_accuracy: 0.8083
Epoch 5/6
9/9 [==============================] - 0s 31ms/step - loss: 0.2278 - accuracy: 0.9170 - val_loss: 0.0970 - val_accuracy: 0.9833
Epoch 6/6
9/9 [==============================] - 0s 29ms/step - loss: 0.0822 - accuracy: 0.9819 - val_loss: 0.0466 - val_accuracy: 0.9917
